# Scraping

Obtener todos los datos a base de Scrapeo:

### Fases
- Con Selenium, obtener todas las urls de los supermercados
- Con Selenium, utilizar las URLs obtenidas, para sacar las urls de:
    - Aceite girasol
    - Aceite de oliva
    - Leche
- Con selenium, sacar las URLs de las subcategorias de:
    - Aceite de oliva
        - Suave e intenso
        - Virgen
        - Virgen extra.
    - Leche
        - Enriquecida
        - Entera, semi o desnatada
        - Sin Lactosa
    De forma que, por supermercado, nos quedaremos con:
    - URL Aceite Girasol
    - URL Aceite de Oliva Suave e Intenso
    - URL Aceite de Oliva Virgen
    - URL Aceite de Oliva Virgen extra
    - URL leche enriquecida
    - URL leche entera,semi o desnatada
    - URL leche sin lactosa

Con estas URL utilizaremos BEAUTIFUL SOUP para sacar las URL del histórico de cada producto

In [6]:
# Importación de librerías necesarias para el proyecto

# Automatización de navegadores (Selenium)
# -----------------------------------------------------------------------
from selenium import webdriver                      # Control automático del navegador para realizar scraping web
from selenium.webdriver.chrome.service import Service  # Manejo del servicio de ChromeDriver
from selenium.webdriver.common.by import By         # Selección de elementos en el DOM
from selenium.webdriver.chrome.options import Options  # Configuración de opciones del navegador (headless mode, etc.)
import time                                         # Gestión de pausas y temporización

# Extracción de datos y scraping
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup                       # Análisis y extracción de datos de HTML y XML (scraping web)

# Tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd                                 # Manipulación y análisis de datos en estructuras DataFrame
import numpy as np                                  # Cálculos numéricos y manejo de arrays
import datetime                                     # Manipulación de fechas y tiempos

# Visualización de progreso
# -----------------------------------------------------------------------
from tqdm import tqdm                               # Creación de barras de progreso durante bucles

# Manejo de tiempo y solicitudes con espera
# -----------------------------------------------------------------------
from time import sleep                              # Pausas en la ejecución del código

# Importación de módulos desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../")                              # Añadir rutas externas para poder importar módulos desde carpetas superiores
from src.SupportScraping import (
    obtener_url_supermercados,
    obtener_url_productos_por_supermercado,
    obtener_urls_por_super_producto_y_categoria,
    obtener_href_productos,
    crear_df_productos_con_historico
)                                                  # Importación de funciones de soporte para scraping

### Obtener con Selenium, obtener todas las urls de los supermercados

In [4]:
xpaths = ["/html/body/section[1]/div/div[2]/div[1]/div/div[2]",
          "/html/body/section[1]/div/div[2]/div[2]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[3]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[4]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[5]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[6]/div/div[2]/div"]

url_supermercados = [obtener_url_supermercados(xpath) for xpath in tqdm(xpaths)]

100%|██████████| 6/6 [01:13<00:00, 12.30s/it]


Observamos el resultado

In [5]:
url_supermercados

['https://super.facua.org/mercadona/',
 'https://super.facua.org/carrefour/',
 'https://super.facua.org/eroski/',
 'https://super.facua.org/dia/',
 'https://super.facua.org/hipercor/',
 'https://super.facua.org/alcampo/']

## Con Selenium, utilizar las URLs obtenidas, para sacar las urls de:
- Aceite girasol
- Aceite de oliva
- Leche

In [26]:
url_productos = [obtener_url_productos_por_supermercado(url) for url in tqdm(url_supermercados)]

100%|██████████| 6/6 [05:14<00:00, 52.36s/it]


In [27]:
url_productos

[['https://super.facua.org/mercadona/aceite-de-girasol/',
  'https://super.facua.org/mercadona/aceite-de-oliva/',
  'https://super.facua.org/mercadona/leche/'],
 ['https://super.facua.org/carrefour/aceite-de-girasol/',
  'https://super.facua.org/carrefour/aceite-de-oliva/',
  'https://super.facua.org/carrefour/leche/'],
 ['https://super.facua.org/eroski/aceite-de-girasol/',
  'https://super.facua.org/eroski/aceite-de-oliva/',
  'https://super.facua.org/eroski/leche/'],
 ['https://super.facua.org/dia/aceite-de-girasol/',
  'https://super.facua.org/dia/aceite-de-oliva/',
  'https://super.facua.org/dia/leche/'],
 ['https://super.facua.org/hipercor/aceite-de-girasol/',
  'https://super.facua.org/hipercor/aceite-de-oliva/',
  'https://super.facua.org/hipercor/leche/'],
 ['https://super.facua.org/alcampo/aceite-de-girasol/',
  'https://super.facua.org/alcampo/aceite-de-oliva/',
  'https://super.facua.org/alcampo/leche/']]

In [33]:
urls_super_category = [obtener_urls_por_super_producto_y_categoria(url_productos[i][0],url_productos[i][1],url_productos[i][2]) for i,p in enumerate(url_productos)]

### Obtener las URL para acceder a las tablas de historico

In [34]:
supermercados_dfs = [obtener_href_productos(df) for df in tqdm(urls_super_category)]



0it [09:07, ?it/s]
0it [07:13, ?it/s]
  0%|          | 0/6 [07:13<?, ?it/s]





100%|██████████| 6/6 [00:17<00:00,  3.00s/it]


### Sacamos la última tabla de la operación anterior que nos unifica todo en una

In [35]:
df = pd.DataFrame()
lista_dfs = []
for i in range(0,len(supermercados_dfs)):
    for e in range(0,len(supermercados_dfs[i])):
        df_temp = supermercados_dfs[i][e]
        df = pd.concat([df,df_temp])
    
    lista_dfs.append(df)

In [36]:
df = lista_dfs[5]

In [37]:
df.sample(6)

,supermercado,categoria,Producto,URL
49,hipercor,leche-entera-semi-desnatada,El Corte Ingles Leche Desnatada De Asturias Br...,https://super.facua.org/hipercor/leche/el-cort...
72,alcampo,leche-entera-semi-desnatada,Möntbelle Leche Fresca De Cabra 750 Ml.,https://super.facua.org/alcampo/leche/mntbelle...
13,hipercor,leche-enriquecida,Gaza Leche Semidesnatada Con Calcio Brik 1 L,https://super.facua.org/hipercor/leche/gaza-le...
1,mercadona,leche-sin-lactosa,Leche Desnatada Sin Lactosa Hacendado 6 L.,https://super.facua.org/mercadona/leche/leche-...
1,mercadona,aceite-de-oliva-suave-e-intenso,Aceite De Oliva Suave Hacendado 3 L.,https://super.facua.org/mercadona/aceite-de-ol...
10,hipercor,aceite-de-oliva-suave-e-intenso,"Carbonell Aceite De Oliva Suave 0,4º Botella 1 L",https://super.facua.org/hipercor/aceite-de-oli...


In [38]:
df.reset_index(drop=True,inplace=True)

In [39]:
df.to_csv("../datos/backups/URLS_historico_productos.csv")

In [3]:
df = pd.read_csv("../datos/backups/URLS_historico_productos.csv",index_col="Unnamed: 0")

In [4]:
df.head()

,supermercado,categoria,Producto,URL
0,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",https://super.facua.org/mercadona/aceite-de-gi...
1,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",https://super.facua.org/mercadona/aceite-de-gi...
2,mercadona,aceite-de-oliva-suave-e-intenso,Aceite De Oliva Intenso Hacendado 3 L.,https://super.facua.org/mercadona/aceite-de-ol...
3,mercadona,aceite-de-oliva-suave-e-intenso,Aceite De Oliva Suave Hacendado 3 L.,https://super.facua.org/mercadona/aceite-de-ol...
4,mercadona,aceite-de-oliva-virgen,Aceite De Oliva Virgen Hacendado 1 L.,https://super.facua.org/mercadona/aceite-de-ol...


In [9]:
len(df)

1531

### A partir de todas las URL sacamos la tabla por producto, día y variación

In [5]:
tablas_historicos = crear_df_productos_con_historico(df)

1531it [05:54,  4.32it/s]


### Observamos que tenemos 1531 Dataframes, 1 por producto

In [11]:
len(tablas_historicos)

1531

### Unimos la lista de dataframes a uno solo

In [6]:
df_historicos = pd.concat(tablas_historicos)

In [10]:
df_historicos.head()

,supermercado,categoria,producto,Día,Precio (€),Variación
0,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",12/07/2024,1.45,=
1,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",13/07/2024,1.45,=
2,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",14/07/2024,1.45,=
3,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",15/07/2024,1.45,=
4,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",16/07/2024,1.45,=


### Lo guardamos para continuar más adelante

In [8]:
df_historicos.to_csv("../datos/backups/02_tabla_completa_historico_productos.csv")

### Lo abrimos aquí, si hiciera falta

In [328]:
df_historicos = pd.read_csv("../datos/backups/02_tabla_completa_historico_productos.csv",index_col="Unnamed: 0")

In [329]:
df_historicos.head()

,supermercado,categoria,producto,Día,Precio (€),Variación
0,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",12/07/2024,1.45,=
1,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",13/07/2024,1.45,=
2,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",14/07/2024,1.45,=
3,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",15/07/2024,1.45,=
4,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",16/07/2024,1.45,=


### Corregir Formatos de la tabla
Vamos a corregir categoría, día, precio y variación
- Categoría: eliminar los guiones
- día: pasarla a formato Datetime
- Precio: pasarla a float
- Variación: Vamos a dejarla como float, pasando:
    - =: convertirlo a 0
    - variaciones, eliminar paréntesís, porcentajes... de formas que quede como + 3.5 o -2.5

### Reemplazar guiones por espacio

In [330]:
df_historicos["categoria"] = df_historicos["categoria"].str.replace("-", " ")

Observamos que hay un espacio innecesario en aceite de girasol

In [331]:
df_historicos["categoria"].unique()

array(['aceite de girasol ', 'aceite de oliva suave e intenso',
       'aceite de oliva virgen', 'aceite de oliva virgen extra',
       'leche enriquecida', 'leche entera semi desnatada',
       'leche sin lactosa'], dtype=object)

Lo eliminamos con rstrip

In [332]:
df_historicos["categoria"] = df_historicos["categoria"].str.rstrip(" ")

In [333]:
df_historicos["categoria"].unique()

array(['aceite de girasol', 'aceite de oliva suave e intenso',
       'aceite de oliva virgen', 'aceite de oliva virgen extra',
       'leche enriquecida', 'leche entera semi desnatada',
       'leche sin lactosa'], dtype=object)

### Pasar el día a formato Datetime

In [334]:
df_historicos["Día"] = pd.to_datetime(df_historicos["Día"], format="%d/%m/%Y")

In [335]:
df_historicos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129427 entries, 0 to 101
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   supermercado  129427 non-null  object        
 1   categoria     129427 non-null  object        
 2   producto      129427 non-null  object        
 3   Día           129427 non-null  datetime64[ns]
 4   Precio (€)    129427 non-null  float64       
 5   Variación     129427 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 6.9+ MB


In [336]:
df_historicos.head()

,supermercado,categoria,producto,Día,Precio (€),Variación
0,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-12,1.45,=
1,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-13,1.45,=
2,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-14,1.45,=
3,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-15,1.45,=
4,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-16,1.45,=


### Precio ya está en float, no repetimos operación

## Arreglar Variación
Vamos a modificar la tabla variación, reemplazando el = por 0

In [337]:
df_historicos["Variación"] = df_historicos["Variación"].str.replace("=","0")

In [338]:
df_historicos.sample()

,supermercado,categoria,producto,Día,Precio (€),Variación
32,hipercor,leche sin lactosa,Larsa Leche Desnatada Sin Lactosa Brik 1 L,2024-08-23,1.18,0


### Modificar los valores que no son 0
Vamos a reemplazar valores como por ejemplo:
- +0,17 (7,23%)

Conviertiendolo en dos columnas
- Variación en €: 0.17
- Variación en %: 7.23

In [339]:
condicion = df_historicos["Variación"] != "0"
df_historicos[condicion].head()

,supermercado,categoria,producto,Día,Precio (€),Variación
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-10-01,1.48,"+0,03 (2,07%)"
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",2024-10-01,6.91,"+0,16 (2,37%)"
9,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-07-01,22.76,"-1,14 (4,77%)"
60,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-08-21,19.85,"-2,91 (12,79%)"
101,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-10-01,20.25,"+0,40 (2,02%)"


### Separamos por el paréntesis

In [340]:
df_historicos[["variación (€)", "variación (%)"]]= df_historicos["Variación"].str.split("(",expand=True)

In [341]:
condicion = df_historicos["Variación"] != "0"
df_historicos[condicion].head()

,supermercado,categoria,producto,Día,Precio (€),Variación,variación (€),variación (%)
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-10-01,1.48,"+0,03 (2,07%)","+0,03","2,07%)"
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",2024-10-01,6.91,"+0,16 (2,37%)","+0,16","2,37%)"
9,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-07-01,22.76,"-1,14 (4,77%)","-1,14","4,77%)"
60,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-08-21,19.85,"-2,91 (12,79%)","-2,91","12,79%)"
101,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-10-01,20.25,"+0,40 (2,02%)","+0,40","2,02%)"


Eliminamos la columna Variación

In [342]:
df_historicos.drop(columns="Variación",inplace=True)

In [343]:
condicion = df_historicos["variación (€)"] != "0"
df_historicos[condicion].head()

,supermercado,categoria,producto,Día,Precio (€),variación (€),variación (%)
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-10-01,1.48,"+0,03","2,07%)"
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",2024-10-01,6.91,"+0,16","2,37%)"
9,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-07-01,22.76,"-1,14","4,77%)"
60,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-08-21,19.85,"-2,91","12,79%)"
101,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-10-01,20.25,"+0,40","2,02%)"


Eliminamos en variacion % el paréntesis, el % y reemplazamos la "," por un "."

In [344]:
df_historicos["variación (%)"] = df_historicos["variación (%)"].str.strip(")").str.replace(",",".")
df_historicos["variación (%)"] = df_historicos["variación (%)"].str.strip("%")

In [345]:
condicion = df_historicos["variación (€)"] != "0"
df_historicos[condicion].head()

,supermercado,categoria,producto,Día,Precio (€),variación (€),variación (%)
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-10-01,1.48,"+0,03",2.07
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",2024-10-01,6.91,"+0,16",2.37
9,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-07-01,22.76,"-1,14",4.77
60,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-08-21,19.85,"-2,91",12.79
101,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-10-01,20.25,"+0,40",2.02


Eliminamos la "," en variacion y ponemos un "."

In [346]:
df_historicos["variación (€)"] = df_historicos["variación (€)"].str.replace(",",".")

In [347]:
condicion = df_historicos["variación (€)"] != "0"
df_historicos[condicion].head()

,supermercado,categoria,producto,Día,Precio (€),variación (€),variación (%)
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-10-01,1.48,+0.03,2.07
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",2024-10-01,6.91,+0.16,2.37
9,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-07-01,22.76,-1.14,4.77
60,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-08-21,19.85,-2.91,12.79
101,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-10-01,20.25,+0.40,2.02


convertimos variacion (€) y variación (%) a float

In [348]:
df_historicos["variación (€)"] = df_historicos["variación (€)"].astype(float)


In [349]:
condicion = df_historicos["variación (€)"] != 0
df_historicos[condicion].head()

,supermercado,categoria,producto,Día,Precio (€),variación (€),variación (%)
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-10-01,1.48,0.03,2.07
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",2024-10-01,6.91,0.16,2.37
9,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-07-01,22.76,-1.14,4.77
60,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-08-21,19.85,-2.91,12.79
101,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-10-01,20.25,0.40,2.02


### Detectamos que la variación va a tener un problema
No podemos saber si el porcentaje es positivo o negativo, para ello, si la variacion es: 

Variación = variacion(€)/(Precio-Variacion) x100

la vamos a calcular

In [350]:
df_historicos["nueva variacion (%)"] = round((df_historicos["variación (€)"] / (df_historicos["Precio (€)"] - df_historicos["variación (€)"])) * 100, 2)

In [351]:
condicion = df_historicos["variación (€)"] != 0
df_historicos[condicion].head()

,supermercado,categoria,producto,Día,Precio (€),variación (€),variación (%),nueva variacion (%)
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-10-01,1.48,0.03,2.07,2.07
81,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",2024-10-01,6.91,0.16,2.37,2.37
9,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-07-01,22.76,-1.14,4.77,-4.77
60,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-08-21,19.85,-2.91,12.79,-12.79
101,mercadona,aceite de oliva suave e intenso,Aceite De Oliva Intenso Hacendado 3 L.,2024-10-01,20.25,0.40,2.02,2.02


Pasamos a float, eliminamos la anterior variacion y renombramos columnas para eliminar tildes y mayúsculas que nos pueden dificultar la búsqueda

In [352]:
df_historicos["nueva variacion (%)"] = df_historicos["nueva variacion (%)"].astype(float)

In [353]:
df_historicos.drop(columns="variación (%)",inplace=True)

In [354]:
df_historicos = df_historicos.rename(columns={"Día": "fecha","Precio (€)":"precio (€)","variación (€)":"variacion (€)","nueva variacion (%)":"variacion (%)"})

In [355]:
df_historicos.head()

,supermercado,categoria,producto,fecha,precio (€),variacion (€),variacion (%)
0,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-12,1.45,0.0,0.0
1,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-13,1.45,0.0,0.0
2,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-14,1.45,0.0,0.0
3,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-15,1.45,0.0,0.0
4,mercadona,aceite de girasol,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",2024-07-16,1.45,0.0,0.0


In [356]:
df_historicos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129427 entries, 0 to 101
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   supermercado   129427 non-null  object        
 1   categoria      129427 non-null  object        
 2   producto       129427 non-null  object        
 3   fecha          129427 non-null  datetime64[ns]
 4   precio (€)     129427 non-null  float64       
 5   variacion (€)  129427 non-null  float64       
 6   variacion (%)  129427 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 7.9+ MB


# Verificar duplicados

Vamos a observar si hay duplicados en nuestra tabla

In [357]:
df_historicos.duplicated().value_counts()

False    128796
True        631
Name: count, dtype: int64

Reseteamos el index, que estaba mal indexado

In [358]:
df_historicos.reset_index(inplace=True,drop=True)

In [360]:
df_historicos[df_historicos.duplicated(keep=False)].sort_values(by=["fecha","producto"])

,supermercado,categoria,producto,fecha,precio (€),variacion (€),variacion (%)
99019,alcampo,aceite de oliva suave e intenso,Carbonell Aceite De Oliva Suave Garrafa De 5 L.,2024-06-22,52.15,0.0,0.0
99144,alcampo,aceite de oliva suave e intenso,Carbonell Aceite De Oliva Suave Garrafa De 5 L.,2024-06-22,52.15,0.0,0.0
99020,alcampo,aceite de oliva suave e intenso,Carbonell Aceite De Oliva Suave Garrafa De 5 L.,2024-06-23,52.15,0.0,0.0
99145,alcampo,aceite de oliva suave e intenso,Carbonell Aceite De Oliva Suave Garrafa De 5 L.,2024-06-23,52.15,0.0,0.0
99021,alcampo,aceite de oliva suave e intenso,Carbonell Aceite De Oliva Suave Garrafa De 5 L.,2024-06-24,52.15,0.0,0.0
...,...,...,...,...,...,...,...
55615,dia,leche sin lactosa,Leche Semidesnatada Sin Lactosa Pascual Brik 1 L,2024-10-26,1.46,0.0,0.0
55719,dia,leche sin lactosa,Leche Semidesnatada Sin Lactosa Pascual Brik 6...,2024-10-26,8.76,0.0,0.0
55823,dia,leche sin lactosa,Leche Semidesnatada Sin Lactosa Pascual Brik 6...,2024-10-26,8.76,0.0,0.0
111079,alcampo,aceite de oliva virgen extra,Ybarra Aceite De Oliva Virgen Extra Botella De...,2024-10-26,11.64,0.0,0.0


### Nota sobre los duplicados:
Al ir a ver los duplicados, observamos que la razón es que en la web, el producto se lista dos veces por ejemplo:

![image.png](../src/01_png/01_Producto_duplicado.png)

Pero si entramos en ambos productos a su histórico: observamos lo siguiente:

![image2.png](../src/01_png/02_URL_es_lo_mismo.png)

![image2.png](../src/01_png/02_URL_es_lo_mismo.png)

La URL es la misma.

Tras observar que es algo que ocurre de forma recurrente, nos permite eliminar los duplicados sin riesgo de perder datos.



# Eliminar duplicados

In [365]:
longitud_inicial = len(df_historicos)
df_historicos = df_historicos.drop_duplicates(keep="first")
longitud_final = len(df_historicos)
print(f"Se han eliminado {longitud_inicial - longitud_final} duplicados")

Se han eliminado 631 duplicados


Reseteamos el index

In [368]:
df_historicos.reset_index(drop=True,inplace=True)

# Guardar la tabla
Guardamos lo conseguido hasta el momento. y procedemos a la preparación preDB

In [369]:
df_historicos.to_csv("../datos/01_tabla_historicos_preDB.csv")